In [ ]:
import mbuild as mb

import utils

# dictionary of SMARTS features using for coarse graining
features = utils.features_dict

for feature, smiles in features.items():
    print(feature, smiles)

Building from the mbuild.Compound class allows us to more easily use already-developed tools! Anything that mbuild can import can be converted to a CG_Compound 

In [ ]:
smiles = "CCCCCCCCC"
mb_nonane = mb.load(smiles, smiles=True)

nonane = utils.CG_Compound()

# This from_mbuild function is very new and only copies the particles and bonds.
# I'd be interested to know where it breaks!
nonane.from_mbuild(mb_nonane)

nonane.visualize().show()

In [ ]:
# convert to pybel mol
mol = nonane.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol.OBMol.PerceiveBondOrders()

cg_nonane = utils.coarse(mol, [features["alkyl_3"]], atomistic=True)

cg_nonane.visualize(color_scheme={"_A": "pink"}).show()

Run these if you want to step through examples of the CG_Compound coarse-graining

In [ ]:
!wget https://bitbucket.org/cmelab/msibi_tests/downloads/P3HT_4-density_0.75-n_compounds_20-traj.gsd

In [ ]:
gsdfile = "P3HT_4-density_0.75-n_compounds_20-traj.gsd"

# Coordinates are scaled from planckton sigma units
scale_factor = 0.356
comp0 = utils.CG_Compound.from_gsd(gsdfile, frame=0, scale=scale_factor)

# pybel will not correctly parse particles with AMBER typing
comp0.amber_to_element()

# unwrap feature won't move particles if the compound doesn't have bonds
# that span the periodic boundary -- note the warning msg
comp0.unwrap()

# to_pybel is from mbuild PR555
mol0 = comp0.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol0.OBMol.PerceiveBondOrders()

In [ ]:
# Notice that the initial frame is typed correctly
# the structure is good so pybel can type it
cg_comp0 = utils.coarse(
    mol0, [features["thiophene"], features["alkyl_3"]], atomistic=True
)


cg_comp0.visualize(color_scheme={"Car": "black", "_A": "pink", "_B": "green"}).show()

In [ ]:
# same process as above but with last frame of trajectory
comp1 = utils.CG_Compound.from_gsd(gsdfile, frame=-1, scale=scale_factor)

comp1.amber_to_element()

# clearly this thing has some pbc problems

comp1.visualize(color_scheme={"Car": "black", "_A": "pink", "_B": "green"}).show()

In [ ]:
# fixed
comp1.unwrap()


comp1.visualize(color_scheme={"Car": "black", "_A": "pink", "_B": "green"}).show()

In [ ]:
mol1 = comp1.to_pybel(box=mb.Box(comp1.box))
mol1.OBMol.PerceiveBondOrders()

# Even with fixing pbc issues, the last frame is distorted enough that
# pybel can't recognise the features (bendy aromatic rings are NO)
cg_comp1 = utils.coarse(
    mol1, [features["thiophene"], features["alkyl_3"]], atomistic=True
)


cg_comp1.visualize(color_scheme={"Car": "black", "_A": "pink", "_B": "green"}).show()

In [ ]:
# But since these are from the same trajectory, they have
# the same number of particles in the same order, so we can
# "fix" the bad morphology using the good one!
mol1_fixed = utils.map_good_on_bad(mol0, mol1)

# Hey look it's fixed =D
cg_comp1_fixed = utils.coarse(
    mol1_fixed, [features["thiophene"], features["alkyl_3"]], atomistic=True
)


cg_comp1_fixed.visualize(
    color_scheme={"Car": "black", "_A": "pink", "_B": "green"}
).show()

# and we can rewrap it into the box
cg_comp1_fixed.wrap()


cg_comp1_fixed.visualize(
    color_scheme={"Car": "black", "_A": "pink", "_B": "green"}
).show()